# Some plots from the Golf model

In [1]:
# Import stuff
import sys, os, time
import pandas as pd
import numpy as np


# Use plotly for plotting
from chart_studio import plotly
#import plotly
import chart_studio.plotly as pltly
import plotly.graph_objs as go
import plotly.offline as pltlyoff
from IPython.display import display, HTML

# Custom files
sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath('__file__')))) # Add parent folder to sys.path
from imports import golf_ballstics

# Setup
pltlyoff.init_notebook_mode(connected=True)
print(sys.version)
#print('Plotly version', plotly.__version__)

# Declare ballistics model
golf_m = golf_ballstics.golf_ballstics()

3.9.4 (tags/v3.9.4:1f2e308, Apr  6 2021, 13:40:21) [MSC v.1928 64 bit (AMD64)]


## The Golf ballistics model

The **drag force** $F_D$ of a ball is given by:
$$F_D=\frac{1}{2}C_D\rho A v^2$$
where $C_D$ is the drag coefficient, $\rho$ is the fluid density (1.2$kg/m^3$ for air), and $A=\pi r^2$ is the cross section area of the ball, and $v$ is the velocity of the ball.
Similarly, the **lift force** created by the _Magnus effect_ (spin induced lift) is given by:
$$F_L=\frac{1}{2}C_L\rho A v^2$$
whre $C_L$ is dependent on the spin.
The ideal lift of a spinning ball is given by:
$$F_{L(ideal)}=\frac{4}{3}4\pi r^3 s \rho v$$
where s is the spin in rad/s.
$C_L$ is the empirically obtained correction factor between the ideal and observed lift, and can be caluclated from the effectivce spin number:
$$s_n=\frac{\omega_n r}{v}$$
with the epirical relation:

| $s_n$ | $C_L$ |
|-------|-------|
| 0.00  | 0.00  |
| 0.04  | 0.10  |
| 0.10  | 0.16  |
| 0.20  | 0.23  |
| 0.40  | 0.33  |


The drag coefficient is dependent on the Reynolds number, which again is dependent on the travelling speeds. We assume that the drag coefficient is constant for the entire ballistic trajectory as a simplification. Reynolds number is the ratio of the inertia forces to the viscous forces given by:
$$Re=\frac{v\rho l}{\mu}$$
where $l$ is a reference length and $\mu$ is the viscocity coefficient.



In [2]:
# Example of how to run the model

##########################################################
########### Simulate entire ball trajectory ##############

# Hit the ball
golf_m.initiate_hit(velocity = 70, 
                    launch_angle_deg = 12, off_center_angle_deg = -8, 
                    spin_rpm = 3000, spin_angle_deg = -10,
                    windspeed = 8, windheading_deg = 100) 

# Simulation results stored in dataframe df
df=golf_m.df_simres 

display(df.head())

##########################################################
########### Calculate landing position ###################

# If we just want to calculate the landing position of the ball 

y2, y1 = golf_m.get_landingpos(velocity = 70, 
                    launch_angle_deg = 12, off_center_angle_deg = -8, 
                    spin_rpm = 3000, spin_angle_deg = -10,
                    windspeed = 8, windheading_deg = 100)

print('Ball lands at ({},{})'.format(y1, y2))


,t,x,y,z,v_x,v_y,v_z
0,0.00000,0.000000,0.000000,0.000000,-9.529228,67.803983,14.553818
1,0.10101,-0.918844,6.713281,1.466234,-8.674823,65.149502,14.468394
2,0.20202,-1.754582,13.167394,2.921163,-7.882605,62.669696,14.331118
3,0.30303,-2.513184,19.379189,4.359816,-7.146759,60.349362,14.145905
4,0.40404,-3.200117,25.364126,5.777315,-6.462731,58.175795,13.913133


Ball lands at (212.69360478310537,8.851091749485533)


# Create plots

In [3]:
### Default data used for all plotting ###
##########################################

# Default input case
dict_default_input = {
    'velocity':70, 
    'launch_angle_deg':12, 
    'spin_rpm':5000, 
    'windspeed':8, 
    'windheading_deg':100, 
    'spin_angle_deg':-10, 
    'off_center_angle_deg':-8}

# Ranges 
dict_ranges = {
    'velocity':(10, 90), 
    'launch_angle_deg':(6, 30), 
    'spin_rpm':(1000, 9000), 
    'windspeed':(0, 8), 
    'windheading_deg':(-90, 270), # [-90, 90] = tail wind
    'spin_angle_deg':(-45, 45), 
    'off_center_angle_deg':(-45, 45)}


### Ball trajectory plot

In [4]:
# Run model
golf_m.initiate_hit(**dict_default_input)
df=golf_m.df_simres

# Remove points where z<0
trace0 = go.Scatter3d(x=df[df['z']>=0]['x'], y=df[df['z']>=0]['y'], z=df[df['z']>=0]['z'],
                        mode='markers',
                        marker = dict(
                            color = 'rgb(255, 255, 255)',
                            size = 3,
                            line = dict(
                                color = 'rgb(150, 150, 150)',
                                width = 1
                            )
                        ),
                        name='trajectory'
                     )

data = [trace0]

abs_x = max(abs(df[df['z']>=0]['x'])) # For plotting
scene=dict(
    xaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(165, 210, 247)',
        range = [-abs_x - 5, abs_x + 5],
        #range = [abs_x + 5, -abs_x - 5],
        title = 'x'
    ),
    yaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(165, 210, 247)',
        range = [0,max(df[df['z']>=0]['y'])*1.1],
        title = 'y'
    ),
    zaxis=dict(
        gridcolor='rgb(255, 255, 255)',
        zerolinecolor='rgb(255, 255, 255)',
        showbackground=True,
        backgroundcolor='rgb(148, 206, 161)',
        range = [0, max(max(df['z']), 1)],
        title = ''
    ),
    aspectratio = dict( x= 1, y= 2.5, z = 1 ),
    #aspectratio = dict( x= 3, y= 1, z = 1 ),
    aspectmode = 'manual',
    camera = {'eye':{'x':-2.2, 'y':0.2, 'z':0.3}}
)


layout=go.Layout(title='', showlegend=False, margin={'t':10}, scene=scene)


fig=go.Figure(data=data, layout=layout)
pltlyoff.iplot(fig, filename='tmp', show_link=False)

y2, y1, err = golf_m.get_landingpos(check = True, **dict_default_input)
print('Input:', dict_default_input)
print('Landing position: ({},{}). Length = {}'.format(y1, y2, np.linalg.norm([y1, y2])))

Input: {'velocity': 70, 'launch_angle_deg': 12, 'spin_rpm': 5000, 'windspeed': 8, 'windheading_deg': 100, 'spin_angle_deg': -10, 'off_center_angle_deg': -8}
Landing position: (216.1086812956981,20.530310393617565). Length = 217.08167996453292


### 2D plot of landing position for one single input

In [5]:
# Input variable
x_name = 'velocity'

# Number of points
n_pts = 100

# Create x values
x_vals = np.linspace(dict_ranges[x_name][0], dict_ranges[x_name][1], n_pts)

# Copy default input
dict_input = dict_default_input.copy()

# Calculate landing position for each x in x_vals
y1_vals = np.zeros(n_pts)
y2_vals = np.zeros(n_pts)
for i in range(n_pts):
    dict_input[x_name] = x_vals[i]
    
    y2, y1 = golf_m.get_landingpos(**dict_input)
    y1_vals[i] = y1
    y2_vals[i] = y2

# Create plots
trace_y1 = go.Scatter(x = x_vals, y = y1_vals, mode = 'lines', name = 'y1')
layout = go.Layout(title = 'Plot of landing position y1', xaxis = dict(title = x_name), yaxis = dict(title = 'y1'))
fig_y1 = go.Figure(data=[trace_y1], layout=layout)

trace_y2 = go.Scatter(x = x_vals, y = y2_vals, mode = 'lines', name = 'y2')
layout = go.Layout(title = 'Plot of landing position y2', xaxis = dict(title = x_name), yaxis = dict(title = 'y2'))
fig_y2 = go.Figure(data=[trace_y2], layout=layout)

pltlyoff.iplot(fig_y1, filename='tmp', show_link=False)
pltlyoff.iplot(fig_y2, filename='tmp', show_link=False)